In [1]:
from models.PCA import PCA
from models.FF import FF

import pandas as pd
import numpy as np
from tqdm import tqdm
from utils import charas
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def model_inference_and_predict(model):
    mon_list = pd.read_pickle('data/mon_list.pkl')
    test_mons = mon_list.loc[mon_list >= model.test_period[0]]
    inference_result = []
    predict_result = []
    T_bar = tqdm(test_mons.groupby(test_mons.apply(lambda x: x//10000)), colour='red', desc=f'{model.name} Inferencing & Predicting')
    
    for g in T_bar: # rolling train
        T_bar.set_postfix({'Year': g[0]})
        model.train_model()
        
        for m in g[1].to_list():
            inference_result.append(model.inference(m))
            predict_result.append(model.predict(m))
        # model refit (change train period and valid period)
        model.refit()

    inference_result = pd.DataFrame(inference_result, index=test_mons, columns=charas)
    inference_result.to_csv(f'results/inference/{model.name}_inference.csv')
    
    predict_result = pd.DataFrame(predict_result, index=test_mons, columns=charas)
    predict_result.to_csv(f'results/predict/{model.name}_predict.csv')

In [ ]:
pca_1 = PCA(K=1, portfolio=True)
pca_2 = PCA(K=2, portfolio=True)
pca_3 = PCA(K=3, portfolio=True)
pca_4 = PCA(K=4, portfolio=True)
pca_5 = PCA(K=5, portfolio=True)
pca_6 = PCA(K=6, portfolio=True)

In [ ]:
model_inference_and_predict(pca_1)
# model_inference_and_predict(pca_2)
# model_inference_and_predict(pca_3)
# model_inference_and_predict(pca_4)
# model_inference_and_predict(pca_5)
# model_inference_and_predict(pca_6)

In [ ]:
ff_1 = FF(K=1, portfolio=True)
ff_2 = FF(K=2, portfolio=True)
ff_3 = FF(K=3, portfolio=True)
ff_4 = FF(K=4, portfolio=True)
ff_5 = FF(K=5, portfolio=True)
ff_6 = FF(K=6, portfolio=True)

In [ ]:
model_inference_and_predict(ff_1)
model_inference_and_predict(ff_2)
model_inference_and_predict(ff_3)
model_inference_and_predict(ff_4)
model_inference_and_predict(ff_5)
model_inference_and_predict(ff_6)

In [5]:
def model_inference_and_predict_CA(model):
    mon_list = pd.read_pickle('data/mon_list.pkl')
    test_mons = mon_list.loc[mon_list >= model.test_period[0]]
    inference_result = pd.DataFrame()
    predict_result = pd.DataFrame()
    T_bar = tqdm(test_mons.groupby(test_mons.apply(lambda x: x//10000)), colour='red', desc=f'{model.name} Inferencing & Predicting')
    
    stock_index = pd.Series(dtype=np.int64)
    for g in T_bar: # rolling train
        T_bar.set_postfix({'Year': g[0]})
        model.train_model()
        
        for m in g[1].to_list():
            m_stock_index, _, _, _ = model._get_item(m)
            stock_index = pd.concat([stock_index, pd.Series(m_stock_index)]).drop_duplicates().astype(int)
            inference_R = model.inference(m) # return (N, 1)
            predict_R = model.inference(m) # reutrn (N, 1)

            # move inference_R and predict_R to cpu
            inference_R = inference_R.cpu().detach().numpy()
            predict_R = predict_R.cpu().detach().numpy()

            inference_R = pd.DataFrame(inference_R, index=m_stock_index, columns=[m])
            predict_R = pd.DataFrame(predict_R, index=m_stock_index, columns=[m])

            
            inference_result = pd.concat([inference_result.reset_index(drop=True), inference_R.reset_index(drop=True)], axis=1) # (N, T)
            predict_result = pd.concat([predict_result.reset_index(drop=True), predict_R.reset_index(drop=True)], axis=1) # (N, T)

            # DEBUG:
            # save inference_R and predict_R to csv
            # inference_result.to_csv(f'temp/{model.name}_inference_stock_{m}.csv')
            # predict_result.to_csv(f'temp/{model.name}_predict_stock_{m}.csv')
            
        # model refit (change train period and valid period)
        model.refit()

    inference_result = pd.DataFrame(inference_result.values, index=charas, columns=test_mons)
    inference_result.to_csv(f'results/inference/{model.name}_inference.csv')
    
    predict_result = pd.DataFrame(predict_result.values, index=charas, columns=test_mons)
    predict_result.to_csv(f'results/predict/{model.name}_predict.csv')
    return inference_result, predict_result

In [6]:
from models.CA import CA0, CA1, CA2, CA3
import gc

In [7]:
ca3_lists = []
infer_results = None
predict_results = None
for i in range(6):
    gc.collect()
    ca3_lists.append(CA3(i + 1).to('cuda'))
    print(f'begin of {ca3_lists[i].name}')
    inference_result, predict_results = model_inference_and_predict_CA(ca3_lists[i])


begin of CA3_1


CA3_1 Inferencing & Predicting: 100%|██████████| 30/30 [01:53<00:00,  3.78s/it, Year=2016]


begin of CA3_2


CA3_2 Inferencing & Predicting:   0%|          | 0/30 [00:02<?, ?it/s, Year=1987]


AssertionError: Dimension mismatch between mon_factor: torch.Size([1, 2]) and mon_beta: torch.Size([94, 2])